---

# Task

### Create a Custom Console Logger

Create a context manager called `ConsoleLogger` that:

1. Redirects all `print` statements within its context to a file
2. Restores normal console output when exiting the context
3. Handles any exceptions that occur and logs them to the file
4. Provides a summary of how many lines were written when exiting

Requirements:

- Use a class-based approach implementing __enter__ and __exit__
- The file name should be provided when creating the context manager
- All prints within the context should go to the file instead of console
- Handle at least 2 exception errors, log them to the file instead of showing
- When exiting, print a summary to the console showing how many lines were logged

Example usage should look like this:
```python
with ConsoleLogger('output.log'):
    print("This should go to the file")
    print("This too")
    # If any error occurs, it should be logged to the file
    x = 2/0 # This should log with 'division by zero' message or something similar.
```

Inside the `output.log`:
```txt
This should go to the file
This too
zero division error
```

In [ ]:
import sys

class ConsoleLogger:
    def __init__(self, file_path):
        self.file_path = file_path
        self.original_stdout = None
        self.file_handle = None
        self.line_count = 0
    
    def __enter__(self):
        self.original_stdout = sys.stdout
        
        self.file_handle = open(self.file_path, 'w')
        
        # Count lines
        class CountingFile:
            def __init__(self, file, counter):
                self.file = file
                self.counter = counter
            
            def write(self, text):
                self.file.write(text)
                self.counter.line_count += text.count('\n')
            
            def flush(self):
                self.file.flush()
        
        # Redirect stdout to counting file
        sys.stdout = CountingFile(self.file_handle, self)
        
        return self
    
    def __exit__(self, exc_type, exc_value, tb):
        
        if exc_type == ZeroDivisionError:   
            error_msg = f"Exception: ZeroDivistionError: {exc_value}"
            self.line_count += 1
            self.file_handle.write(error_msg)
        elif exc_type == TypeError:          
            error_msg = f"Exception: TypeError: {exc_value}"
            self.line_count += 1
            self.file_handle.write(error_msg)
        elif exc_type is not None:             
            error_msg = f"Exception: {exc_type.__name__}: {exc_value}"
            self.line_count += 1
            self.file_handle.write(error_msg)
        
        # Restore original stdout
        sys.stdout = self.original_stdout

        self.file_handle.close()
    
        print(f"{self.line_count} lines were written to {self.file_path}")
        
        return True

In [ ]:
with ConsoleLogger('output.log'):
    print("This should go to the file")
    print("This too")
    x = 2/0

3 lines were written to output.log
